# ML key prediction project
Before we turn to the heavy gears of ML, let us first try to understand the data.

## Understanding 1 track analysis

Import a track analysis file

In [5]:
from pickle import load
from data import get_track_path

def load_extracted_analysis(output_dir, track_id):
    path = get_track_path(output_dir, track_id)
    with open(path, 'rb') as f:
        return load(f)

track_id = '39C5FuZ8C8M0QI8CrMsPkR'
x = load_extracted_analysis('../example/', track_id)
x.keys()

dict_keys(['id', 'segments', 'duration', 'key'])

In [22]:
segments = x['segments']
duration = x['duration']
key = x['key']

In [23]:
f"song lasts {duration} seconds"

'song lasts 467.64 seconds'

[Foreplay / Long Time by Boston](https://open.spotify.com/track/39C5FuZ8C8M0QI8CrMsPkR)

![Album Cover](https://i.scdn.co/image/ab67616d0000485190ef9760e57fafd8f966d17a)

### Musical key.
From Spotify API documentation:

_The estimated overall key of the section. The values in this field ranging from 0 to 11 mapping to pitches using standard Pitch Class notation (E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on). If no key was detected, the value is -1._


In [24]:
key

5

Which we can map to pitch classes using this table: (see [this page](https://en.wikipedia.org/wiki/Pitch_class))



In [3]:
pitch_classes = {
    0: 'C (also B♯, D#)',
    1: 'C♯, D♭ (also B#)',
    2: 'D (also C#, E#)',
    3: 'D♯, E♭ (also F#)',
    4: 'E (also D#, F♭)',
    5: 'F (also E♯, G#)',
    6: 'F♯, G♭ (also E#)',
    7: 'G (also F#, A#)',
    8: 'G♯, A♭',
    9: 'A (also G#, B#)',
    10: 'A♯, B♭ (also C#)',
    11: 'B (also A#, C♭)'
}

In [37]:
pitch_classes[key]

'F (also E♯, G#)'

### Segments

In [30]:
len(segments)

1517

In [33]:
segments[1]

{'start': 0.11279,
 'duration': 0.2239,
 'confidence': 1.0,
 'pitches': [0.617,
  1.0,
  0.165,
  0.129,
  0.236,
  0.384,
  0.134,
  0.075,
  0.203,
  0.074,
  0.055,
  0.389]}

... of which '`pitches`' is the [_Chroma vector_](https://en.wikipedia.org/wiki/Chroma_feature), quoting the Spotify API docs:

_A “chroma” vector representing the pitch content of the segment, corresponding to the 12 pitch classes C, C#, D to B, with values ranging from 0 to 1 that describe the relative dominance of every pitch in the chromatic scale. _

In [4]:
dfs = []
for segment in segments:
    df = pd.DataFrame(segment)
    df = df.rename(columns={'pitches': 'pitch'})
    df['chroma'] = pitch_classes.values()
    dfs.append(df)
df = pd.concat(dfs)
df

NameError: name 'segments' is not defined

In [ ]:
df.to_csv('../example/39C5FuZ8C8M0QI8CrMsPkR.csv', index=False)